In [113]:
# project description -- so what this project will do is:
# 1) load in the ProPublica dataset of traffic tickets for Chicago 
# 2) format the data so that it is usable -- zipcode in particular needs formatting as it is zip+4 in some 
# places, 5 digit zip in others
# 3) run a heatmap showing the traffic ticket "hot spots"
# 4) run an analysis on the violation codes, looking specifically for those involving bike lane obstruction 


# future plans
# * load in the data on 311 complaints about blocked bike lanes 
# correlate the traffic tickets issued 

In [114]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
# BLU-Metis-Project/data/ProPublica data/processed
# read in the raw data

# 1) load in the ProPublica dataset of traffic tickets for Chicago -- note that Github will only take a limited size
# sample, so we call the sample file not the whole thing

PPTicketData = pd.read_csv('data/ProPublica data/processed/parking_tickets_sample.csv', sep=',')
#PPTicketData.to_csv(r'data/ProPublica data/processed/parking_tickets_sample.csv')

In [115]:
# take a look at what you've got
PPTicketData.head(5)

Unnamed: 0  ticket_number           issue_date violation_location  \
0           0       51551278  2007-01-01 00:00:00     6014 W 64TH ST   
1           1       51491256  2007-01-01 00:00:00     530 N MICHIGAN   
2           2       50433524  2007-01-01 00:01:00        4001 N LONG   
3           3       51430906  2007-01-01 00:01:00       303 E WACKER   
4           4       51507779  2007-01-01 00:01:00        7 E 41ST ST   

                                license_plate_number license_plate_state  \
0  90ad622c3274c9bdc9d8c812b79a01d0aaf7479f2bd743...                  IL   
1  bce4dc26b2c96965380cb2b838cdbb95632b7b57160612...                  IL   
2  44641e828f4d894c883c07c566063c2d99d08f2c03b3d4...                  IL   
3  eee50ca0d9be2debd0e7d45bad05b8674a6cf5b892230f...                  IL   
4  244116ca3eed4235b1f61f6d753d8c688be2a48c9fdd97...                  IL   

  license_plate_type    zipcode violation_code  \
0                PAS      60638       0976160F   
1                PAS  606343801       0964150B   
2                PAS      60148       0976160F   
3                PAS      60601       0964110A   
4                PAS  605053013       0976220B   

                      violation_description  ...  fine_level2_amount  \
0  EXPIRED PLATES OR TEMPORARY REGISTRATION  ...                 100   
1       PARKING/STANDING PROHIBITED ANYTIME  ...                 100   
2  EXPIRED PLATES OR TEMPORARY REGISTRATION  ...                 100   
3                DOUBLE PARKING OR STANDING  ...                 200   
4     SMOKED/TINTED WINDOWS PARKED/STANDING  ...                  50   

  current_amount_due total_payments  ticket_queue    ticket_queue_date  \
0                0.0          100.0          Paid  2007-05-21 00:00:00   
1               50.0            0.0        Define  2007-01-22 00:00:00   
2                0.0           50.0          Paid  2007-01-31 00:00:00   
3                0.0          100.0          Paid  2007-03-08 00:00:00   
4                0.0           50.0          Paid  2007-08-29 00:00:00   

   notice_level  hearing_disposition notice_number officer  \
0          SEIZ                  NaN    5048648030   15227   
1           NaN                  NaN             0   18320   
2          VIOL                  NaN    5079875240    3207   
3          DETR               Liable    5023379950   19410   
4          SEIZ                  NaN    5079891400   66396   

                       address  
0  6000 w 64th st, chicago, il  
1  500 n michigan, chicago, il  
2     4000 n long, chicago, il  
3    300 e wacker, chicago, il  
4     7 e 41st st, chicago, il  

[5 rows x 24 columns]

In [116]:
#PPTicketData.dtypes

In [117]:
PPTicketData['FormatIssueDate'] = pd.to_datetime(PPTicketData.issue_date) # the other columns also need formatting
PPTicketData['violation_location'] = PPTicketData['violation_location'].astype('|S') 
PPTicketData['zipcode'] = PPTicketData['zipcode'].astype('str') 

In [118]:
# need to format zipcode
PPTicketData.dtypes

Unnamed: 0                        int64
ticket_number                     int64
issue_date                       object
violation_location               object
license_plate_number             object
license_plate_state              object
license_plate_type               object
zipcode                          object
violation_code                   object
violation_description            object
unit                              int64
unit_description                 object
vehicle_make                     object
fine_level1_amount                int64
fine_level2_amount                int64
current_amount_due              float64
total_payments                  float64
ticket_queue                     object
ticket_queue_date                object
notice_level                     object
hearing_disposition              object
notice_number                     int64
officer                          object
address                          object
FormatIssueDate          datetime64[ns]


In [119]:
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
search = SearchEngine(simple_zipcode=True) # this is 9MB - it's not a good solution

In [120]:
PPTicketData.zipcode.dtypes

dtype('O')

In [121]:
PPTicketData.zipcode
#PPTicketData['FormattedZip'] = PPTicketData.zipcode.str.slice(0,5) # this is where I'm at 8/10 -- how do 
#df['FormattedZip'] = df['zipcode'].str.replace('/',' ')

0           60638
1       606343801
2           60148
3           60601
4       605053013
5           60632
6           60651
7           60643
8           60651
9           60651
10      606532213
11      601481629
12            nan
13      606532213
14      618204259
15          60148
16      601762321
17          60639
18      606251705
19            nan
20          60005
21      618013705
22          60616
23      606345108
24      608041538
25          60076
26      606405542
27          60640
28      605043124
29      606385519
          ...    
1970    604094613
1971          nan
1972        52245
1973        60181
1974          nan
1975        60629
1976        60438
1977    606392837
1978          nan
1979          nan
1980        60026
1981        60631
1982        49093
1983        60628
1984          nan
1985    606287017
1986          nan
1987    606533611
1988    606404214
1989    601877832
1990        60626
1991          nan
1992          nan
1993        60433
1994      

In [122]:
#PPTicketData['FormattedZip'].astype(str) 

In [123]:
#PPTicketData.FormattedZip.head(5)

In [124]:
PPTicketData.zipcode.dtypes

dtype('O')

In [125]:
PPTicketData.zipcode.dropna #doesn't work

<bound method Series.dropna of 0           60638
1       606343801
2           60148
3           60601
4       605053013
5           60632
6           60651
7           60643
8           60651
9           60651
10      606532213
11      601481629
12            nan
13      606532213
14      618204259
15          60148
16      601762321
17          60639
18      606251705
19            nan
20          60005
21      618013705
22          60616
23      606345108
24      608041538
25          60076
26      606405542
27          60640
28      605043124
29      606385519
          ...    
1970    604094613
1971          nan
1972        52245
1973        60181
1974          nan
1975        60629
1976        60438
1977    606392837
1978          nan
1979          nan
1980        60026
1981        60631
1982        49093
1983        60628
1984          nan
1985    606287017
1986          nan
1987    606533611
1988    606404214
1989    601877832
1990        60626
1991          nan
1992          n

In [126]:
#with ZipcodeSearchEngine() as search: 
#cityzipcodes = search.by_city("Chicago")
# unique city zip codes in ProPublica dataset
PPTicketData['FormattedZip'] = PPTicketData.zipcode.str.slice(0,5)
uniqueZipCodesCount = PPTicketData.FormattedZip.unique().shape[0] # I don't actually need this -- can use a for loop tha tincreements the elements
uniqueZipCodes = PPTicketData.FormattedZip.unique()

In [127]:
print(uniqueZipCodesCount)

323


In [128]:
uniqueZipCodes

array(['60638', '60634', '60148', '60601', '60505', '60632', '60651',
       '60643', '60653', 'nan', '61820', '60176', '60639', '60625',
       '60005', '61801', '60616', '60804', '60076', '60640', '60504',
       '60617', '60652', '60649', '60644', '60618', '62411', '60629',
       '89149', '60453', '60115', '60101', '60628', '60165', '60623',
       '60131', '60622', '60637', '60120', '60008', '60624', '60402',
       '60613', '60609', '60193', '60626', '55343', '60630', '60031',
       '60657', '60611', '60440', '60044', '61107', '60610', '60104',
       '60103', '60047', '60181', '60659', '78245', '60160', '60608',
       '60645', '60064', '60016', '60419', '49464', '60426', '60563',
       '60666', '60527', '60153', '62704', '60641', '60646', '60403',
       '60471', '60712', '60139', '60647', '60614', '48306', '60068',
       '60062', '60133', '60612', '60707', '60202', '60171', '60194',
       '60164', '60018', '60409', '60620', '60172', '97140', '76087',
       '60636', '50613

In [129]:
uniqueZipCodes[9]

'nan'

In [130]:
uniqueZipCodes[0]

'60638'

In [131]:
#for i in range(1, uniqueZipCodesCount+1)
#    print(search.by_zipcode(uniqueZipCodes[i]))

In [132]:
zipcodes2 = search.by_zipcode(format(str(uniqueZipCodes[142])))

In [133]:
zipcodes2 = search.by_zipcode(uniqueZipCodes[71])

In [134]:
print(zipcodes2)

SimpleZipcode(zipcode='60527', zipcode_type='Standard', major_city='Willowbrook', post_office_city='Willowbrook, IL', common_city_list=['Willowbrook', 'Burr Ridge'], county='DuPage County', state='IL', lat=41.74, lng=-87.95, timezone='Central', radius_in_miles=5.0, area_code_list=['630', '708'], population=27486, population_density=1898.0, land_area_in_sqmi=14.48, water_area_in_sqmi=0.36, housing_units=12151, occupied_housing_units=11219, median_home_value=395000, median_household_income=72950, bounds_west=-87.973042, bounds_east=-87.895256, bounds_north=41.789615, bounds_south=41.701035)


In [135]:
zipcodes2.zipcode_type != None

True

In [136]:
zipcodes2.lat == None

False

In [137]:
zipcodes2.lng

-87.95

In [138]:
zipcodes2.common_city_list[1]

'Burr Ridge'

In [139]:
#df = pd.read_csv('data/Zip_Codes.csv', sep=',') # do i even need this???
#df.head()

In [140]:
# test case
zipcodes2 = search.by_zipcode(uniqueZipCodes[0])
m = folium.Map([zipcodes2.lat, zipcodes2.lng], marker=zipcodes2.common_city_list[-1], zoom_start=15) # picked at random one location in Chicago
folium.Marker([zipcodes2.lat, zipcodes2.lng], popup=zipcodes2.common_city_list[-1]).add_to(m)
m


In [141]:
zipcodes2.common_city_list

['Chicago', 'Bedford Park']

In [142]:
m2 = folium.Map([zipcodes2.lat, zipcodes2.lng], marker=zipcodes2.common_city_list[-1], zoom_start=12) # picked at random one location in Chicago

for i in range(0, uniqueZipCodesCount):
    #print(uniqueZipCodes[i])
    zipcodes2 = search.by_zipcode(uniqueZipCodes[i])
    if (zipcodes2.lat != None): # check for bad data
        folium.Marker([zipcodes2.lat, zipcodes2.lng], popup=zipcodes2.common_city_list[-1], marker=zipcodes2.common_city_list[-1]).add_to(m2)
m2    

In [143]:
# 3) run a heatmap showing the traffic ticket "hot spots" by owners registration zip code
# subtask 1: find the count for all traffic tickets registered in a sample zip code -- if you have 
# zipcode, you only need to call it once to find the lat and long and save it.  

# create an empty dataframe to hold the zipcode, lat and long
LatLongDF = pd.DataFrame({'zipcode':[], 'Latitude':[], 'Longitude':[]})
LatLongDF['zipcode'] = uniqueZipCodes # now add the uniqueZipCodes data to fill it in

for i in range(0, uniqueZipCodesCount):
    #print(i)
    zipcodes2 = search.by_zipcode(uniqueZipCodes[i])
    if (zipcodes2.lat != None): # check for bad data
        LatLongDF['zipcode'][i] = uniqueZipCodes[i]
        LatLongDF['Latitude'][i] = zipcodes2.lat
        LatLongDF['Longitude'][i] = zipcodes2.lng



/Users/troust/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/troust/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/troust/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [144]:
LatLongDF

zipcode  Latitude  Longitude
0     60638     41.78    -87.770
1     60634     41.95    -87.810
2     60148     41.88    -88.020
3     60601     41.88    -87.620
4     60505     41.76    -88.300
5     60632     41.82    -87.690
6     60651     41.90    -87.740
7     60643     41.70    -87.660
8     60653     41.82    -87.610
9       nan       NaN        NaN
10    61820     40.11    -88.240
11    60176     41.96    -87.870
12    60639     41.92    -87.760
13    60625     41.97    -87.700
14    60005     42.06    -87.980
15    61801     40.11    -88.210
16    60616     41.84    -87.620
17    60804     41.84    -87.760
18    60076     42.03    -87.740
19    60640     41.97    -87.660
20    60504     41.75    -88.250
21    60617     41.73    -87.550
22    60652     41.75    -87.710
23    60649     41.77    -87.560
24    60644     41.88    -87.760
25    60618     41.95    -87.700
26    62411     39.10    -88.700
27    60629     41.78    -87.700
28    89149     36.28   -115.290
29    60453     41.71    -87.760
..      ...       ...        ...
293   60423     41.48    -87.840
294   60477     41.57    -87.780
295   60487     41.57    -87.830
296   60525     41.78    -87.850
297   04102     43.66    -70.300
298   60404     41.51    -88.220
299   53218     43.12    -88.000
300   60603     41.88    -87.629
301   53910     43.90    -89.800
302   60914     41.20    -87.900
303   60102     42.16    -88.310
304   60523     41.84    -87.950
305   17402     39.95    -76.670
306   60513     41.83    -87.850
307   60457     41.72    -87.830
308   60015     42.17    -87.880
309   48359     42.72    -83.280
310   60465     41.70    -87.830
311   44102     41.47    -81.740
312   60522       NaN        NaN
313   53215     43.01    -87.940
314   46321     41.55    -87.500
315   07114     40.70    -74.170
316   60430     41.56    -87.660
317   72127     35.20    -92.600
318   61920     39.50    -88.200
319   50314     41.60    -93.630
320   52245     41.67    -91.510
321   49093     42.00    -85.700
322   60433     41.50    -88.040

[323 rows x 3 columns]

In [145]:
# subtask 1: find the count for all traffic tickets registered in a sample zip code -- if you have 
# zipcode, you only need to call it once to find the lat and long and save it.  
NewPPTicketData = PPTicketData.groupby(['FormattedZip']).size().reset_index(name='counts')

In [146]:
NewPPTicketData.dtypes

FormattedZip    object
counts           int64
dtype: object

In [147]:
LatLongDF.dtypes

zipcode       object
Latitude     float64
Longitude    float64
dtype: object

In [148]:
LatLongDF.rename(columns = {'zipcode':'FormattedZip'}, inplace = True)

In [149]:
LatLongDF.shape

(323, 3)

In [150]:
NewPPTicketData.shape

(323, 2)

In [151]:
NewPPTicketData.dtypes

FormattedZip    object
counts           int64
dtype: object

In [162]:
LatLongDF.dtypes

FormattedZip     object
Latitude        float64
Longitude       float64
dtype: object

In [163]:
# join count to the LatLongDF - now you have a dataframe with the zipcode, lat, long and counts

result = pd.merge(LatLongDF, NewPPTicketData[['FormattedZip', 'counts']], on=['FormattedZip'])
print(result)


    FormattedZip  Latitude  Longitude  counts
0          60638     41.78    -87.770      16
1          60634     41.95    -87.810      12
2          60148     41.88    -88.020      14
3          60601     41.88    -87.620       1
4          60505     41.76    -88.300       5
5          60632     41.82    -87.690      30
6          60651     41.90    -87.740      31
7          60643     41.70    -87.660      16
8          60653     41.82    -87.610      19
9            nan       NaN        NaN     350
10         61820     40.11    -88.240       2
11         60176     41.96    -87.870       6
12         60639     41.92    -87.760      43
13         60625     41.97    -87.700      60
14         60005     42.06    -87.980       2
15         61801     40.11    -88.210       1
16         60616     41.84    -87.620      21
17         60804     41.84    -87.760      15
18         60076     42.03    -87.740       5
19         60640     41.97    -87.660      45
20         60504     41.75    -88.

In [180]:
not np.isnan(result.Latitude[1]) # test case

True

In [230]:
# subtask 2: heatmap creation:
m3 = folium.Map([result.Latitude[0], result.Longitude[0]], zoom_start=11, prefer_canvas=True) # start at the first point
for i in range(0, uniqueZipCodesCount):
   
    if (not np.isnan(result.Latitude[i])): # check for bad data
        #print(i) # test case
        folium.Marker([result.Latitude[i], result.Longitude[i]],
                         marker=zipcodes2.common_city_list[-1]).add_to(m3)
        

In [231]:
# convert to (n, 2) nd-array format for heatmap
stationArr = result[['Latitude', 'Longitude']].as_matrix() # isn't this already a matrix?

/Users/troust/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [232]:
stationArr = stationArr[~np.isnan(stationArr).any(axis=1)] # remove NaNs

In [235]:
# plot heatmap
m3.add_child(plugins.HeatMap(stationArr, radius=30, max_opacity=0.8))
m3.save('heatmap20190815.html')

In [236]:
#from folium.plugins import HeatMap
#map_traffic_tickets = folium.Map(location=[result.Latitude[0], result.Longitude[0]],
#                    zoom_start = 13) 

#heat_data = [[result['Latitude'],result['Longitude']] for index, row in result.iterrows()]
#HeatMap(heat_data).add_to(map_traffic_tickets)


In [ ]:
# 4) run an analysis on the violation codes, looking specifically for those involving bike lane obstruction 

